<a href="https://colab.research.google.com/github/sgomiz/GEE/blob/master/NDVI_2_Dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Example for getting  NDVI values in a pandas data frame by means of a MODIS satellite image collection

---
**Author:** Sergio Gómiz Morcillo
**Versión:** 0.1
**Date:** July 5, 2019

**Description:**

Following this example we can obtain the NDVI (Index of Vegetation in Normarized Differences) values for a specific data range and region of interest. These data series come from a collection of images from the MODIS satellite (**MOD IS/MCD43A4_006_NDVI**)
To get this collection of images. It is necessary to have previously an active GEE account.

This example use both geometry possibilites: polygon or point (coordinates) and it should be like the region parameter to the **NDVI2DF function**.

Considerations to bear in mind: GEE API have strengh limitations about size of result dataset (getRegion function) . **Raised** error . **Too many values**: 42622 points x 10 bands x 685 images > ***1048576***

In [0]:
# Show ROI over the map
from IPython.display import Image
Image(filename="ROI_Cadiz.PNG")

# Prerrequisite: Google Earth Engine API

In [0]:
# Installing GEE API
!pip install earthengine-api

In [0]:
# Getting the access token to GEE API
!earthengine authenticate

In [0]:
import ee, datetime
import numpy as np
import pandas as pd

# Initalization of Google Earth Engine API
ee.Initialize()

# Function NDVI2DF
Own function to encapsulate all core coding, in order to get to the MODIS images collection (**MODIS/MCD43A4_006_NDVI**) and to get the NDVI data series.

In [0]:
def  NDVI2DF(coordinates, region, date_start, end_date):
  #Get the NDVI (Normalised Difference Vegetation Index) from MODIS Satellite via Google Earth Engine
  #coordinates : array of coordinates long/lat
  #region : polygon or point values
  #date_start : Starting Date for setup date range
  #end_date : Ending Date for setup date range 
 
  # Define the type of geometry for setting properly the region coordinates
  if region == 'polygon' :
      ROI = ee.Geometry.Polygon(coordinates)
  elif region == 'point' :  
      ROI = ee.Geometry.Point(coordinates)
  else:
      raise KeyError('Wrong region value, allowed values are polygon or point')


  # Define the image (scala 1000)
  img = ee.ImageCollection('MODIS/MCD43A4_006_NDVI').filterDate(date_start, end_date)
  result = img.select('NDVI').getRegion(ROI,1000).getInfo()


  # Export to dataframe
  df_NDVI = pd.DataFrame(result[1:])
  df_NDVI.columns = result[0]
  
  return df_NDVI


# Testing Function Code
Coordinates define a Region Of Interest (ROI) to collect some data series from a specific region over the map.

In [45]:
# Main Program Code to call function NDVI2DF

# Gettting Date Range
date_start = ee.Date('2019-03-01')
end_date = ee.Date('2019-06-30')

# Setting type of geometry for region of interest (point or polygon)
region = 'point'

# Getting ROI through array of coordinates (point of polygon)
coordinates = [6.134136, 49.612485]
#coordinates = [[-6.151065585896504, 36.29423686205435],
#          [-6.035709140584004, 36.19012306037972],
#          [-5.601749179646504, 36.00370222379312],
#          [-5.313358066365254, 36.21228654591974],
#          [-5.516605136677754, 36.87866023265441],
#          [-6.428470371052754, 36.73131853399765]]

df_result = NDVI2DF (coordinates, region, date_start, end_date)



(114, 5)